# Quantum circuit utils

In [ ]:
#| default_exp platform.qcircuit_util

In [ ]:
#| export
from genQC.imports import *

## SRV

In [ ]:
#| export
def get_element_matching_indices(a, b):
    """Compares (2d) `a` with `b`. Returns the indices of `b`, where a element of `a` matches with `b`."""
    # Expand dimensions of a to match the shape of b for element-wise comparison
    expanded_a = a.unsqueeze(0).expand(b.shape[0], *a.shape)  # [b0, a0, a1]
    expanded_b = b.unsqueeze(1)                               # [b0,  1, b1]
    
    # Compare all vector entries of a with all vectors of b
    matches = torch.all(expanded_a == expanded_b, dim=-1)

    matching_indices = torch.nonzero(torch.any(matches, dim=1)).squeeze()
    
    if   matching_indices.dim() == 0: matching_indices = torch.tensor([matching_indices])

    return matching_indices

In [ ]:
#| export
def get_entanglement_bins(num_of_qubits):
    """Returns all SRV sorted in entangle bins which correspond to a number of entangled qubits."""
    dist_srvs = [x for x in itertools.product(*([[1,2]]*num_of_qubits))]
    dist_srvs = np.array(dist_srvs, dtype=int)[np.sum(dist_srvs, axis=1)!=num_of_qubits+1].tolist()
    dist_srvs = sorted(dist_srvs, key=lambda x: sum(x))
    dist_srvs = np.array(dist_srvs)
    
    entangle = [1] + [scipy.special.comb(num_of_qubits, i, exact=True) for i in range(2, num_of_qubits)]
    
    entanglement_bins = np.split(dist_srvs, np.cumsum(entangle))
    
    ent_bits = [f"{sum(n[0])-num_of_qubits} qubit entangled" for n in entanglement_bins]
    
    return [x.tolist() for x in entanglement_bins], ent_bits

In [ ]:
for srvs,label in zip(*get_entanglement_bins(5)):
    print(label, ":", srvs)

0 qubit entangled : [[1, 1, 1, 1, 1]]
2 qubit entangled : [[1, 1, 1, 2, 2], [1, 1, 2, 1, 2], [1, 1, 2, 2, 1], [1, 2, 1, 1, 2], [1, 2, 1, 2, 1], [1, 2, 2, 1, 1], [2, 1, 1, 1, 2], [2, 1, 1, 2, 1], [2, 1, 2, 1, 1], [2, 2, 1, 1, 1]]
3 qubit entangled : [[1, 1, 2, 2, 2], [1, 2, 1, 2, 2], [1, 2, 2, 1, 2], [1, 2, 2, 2, 1], [2, 1, 1, 2, 2], [2, 1, 2, 1, 2], [2, 1, 2, 2, 1], [2, 2, 1, 1, 2], [2, 2, 1, 2, 1], [2, 2, 2, 1, 1]]
4 qubit entangled : [[1, 2, 2, 2, 2], [2, 1, 2, 2, 2], [2, 2, 1, 2, 2], [2, 2, 2, 1, 2], [2, 2, 2, 2, 1]]
5 qubit entangled : [[2, 2, 2, 2, 2]]


# Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()